# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

When solving this task, we expect you'll face (and successfully deal with) some problems or make up the ideas of the model improvement. Some of them are: 

- solving a problem of n-grams frequencies storing for a large corpus;
- taking into account keyboard layout and associated misspellings;
- efficiency improvement to make the solution faster;
- ...

Please don't forget to describe such cases, and what you decided to do with them, in the Justification section.

##### IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

---

# Norvig's solution

I checked the Norvig's solution and got several points from there (it's actually a great point for the beginning). To see how good it is I copied the code for the further testing and the Norvig's dataset 'big.txt'.

In [1]:
import re
from collections import Counter

def get_words(text): return re.findall(r'\w+', text.lower())

file = open('useful data/big.txt').read()
WORDS = Counter(get_words(file))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]).union(known(edits1(word))) or known(edits2(word)) or [word])

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

---


In [2]:
# form the dictionaries: bigrams, threegrams, fourgrams, fivegrams, and pos_grams
# each dictionary has keys with possible n_grams

bigrams = {}
threegrams = {}
fourgrams = {}
fivegrams = {}
pos_bigrams = {}

# generate 3-4-5 grams from the fivegrams.txt
def generate_n_grams(line: str):
    parts = line.split()
    freq = int(parts[0])
    fivegram = parts[1:]
    for i in range(3):
        three_words = (fivegram[i], fivegram[i+1], fivegram[i+2])
        if three_words in threegrams.keys():
            threegrams[three_words] += freq
        else:
            threegrams[three_words] = freq
    for i in range(2):
        four_words = (fivegram[i], fivegram[i+1], fivegram[i+2], fivegram[i+3])
        if four_words in fourgrams.keys():
            fourgrams[four_words] += freq
        else:
            fourgrams[four_words] = freq
    fivegram = tuple(fivegram)
    if fivegram in fivegrams.keys():
        fivegrams[(fivegram)] += freq
    else:
        fivegrams[(fivegram)] = freq

# generate bigrams fromt he bigrams.txt
def generate_2_grams(line: str):
    parts = line.split()
    two_words = (parts[1], parts[2])
    if two_words in bigrams.keys():
        bigrams[two_words] += int(parts[0])
    else:
        bigrams[two_words] = int(parts[0])

# apply functions
filename = 'useful data/fivegrams.txt'
with open(filename, 'r', encoding='latin-1') as f:
    for line in f:
        generate_n_grams(line)
filename = 'useful data/bigrams.txt'
with open(filename, 'r', encoding='latin-1') as f:
    for line in f:
        generate_2_grams(line)

# also extract bigrams woth pos tags from the coca_all_links.txt
with open('useful data/coca_all_links.txt', 'r') as f:
    for line in f:
        parts = line.split()
        freq, w1, w2, pos1, pos2 = parts
        tup = (w1, pos1, w2, pos2)
        if tup in pos_bigrams.keys():
            pos_bigrams[tup] += int(freq)
        else:
            pos_bigrams[tup] = int(freq)


In [3]:
# define the class with my spell corrector
import spacy

class Corrector():
    def __init__(self,):
        """ Init method. Calls the method init_vars to initialize variables """
        self.init_vars()

    def init_vars(self):
        self.nlp = spacy.load("en_core_web_sm")
        self.log = []
        self.result = []
        self.words = None
        self.tagged = None

    def correct_text(self, text: str, return_list: bool=True) -> list[str]:
        """ Method for correction of the whole text with several sentences
        Args:
            text (str): text splitted on sentences by '.'
        Returns:
            result (list or str): corrected text. The return type depends on
            return_list variable. If True => return list of corrected sentences.
            If False => return the text in the str format.
        """
        result = []
        text = text.split('. ')
        for sentence in text:
            # split the text into sentences and correct them one by one
            result.append(self.correct_sentence(sentence))

        if return_list:
            return result
        return '. '.join(result)
    

    def correct_sentence(self, sentence: str) -> str:
        """ Correction of a passed sentence.
        Args:
            sentence (str): sentence with misspelled words
        Returns:
            self.result (str): corrected sentence
        """
        
        # split the sentence to words and tag them to determine expected tags
        self.words = get_words(sentence)
        self.tagged = self.nlp(sentence)
        self.result = []

        for i, word in enumerate(self.words):
            # check the correctness of each word

            if word in WORDS:
                # if the current word exists in the vocabulary
                # add it to the result and skip
                self.result.append(word)
                continue
            
            # identify the context of the current word
            # (all previous tokens)
            context = self.words[:i]

            # if it is teh 1st word, then context is nothing
            # apply candidates generation and check the pos tags compatibility
            if not context:
                self.check_1st_word(word)
                continue

            # otherwise word is incorrect and isn't the 1st one
            expected_tag = self.tagged[i].dep_

            # get available variants and sort them by frequency in the corpus
            variants = self.get_variants(word)
            variants = sorted(list(variants), key=P, reverse=True)
            
            # if there is no available candidates we cannot choose and
            # paste the same incorrect word
            if not variants:
                self.result.append(word)
                continue

            
            # 1. for each variant we compute some statistics: in how many n_grams types the current context+variant is met 
            # and how many times it is met (using n_grams dictionaries)
            # variables: types_of_grams and occurences
            # 2.1. if the current context+variant cannot be found in any n_grams
            # (types_of_grams = 0), we ignore it for now
            # 2.2. if the current context+variant is found, check if it is the best suitable variant for this context:
            # if types_of_grams > max_gram: save max_gram, max_occurences and the index of this variant
            # if types_of_grams == max_gram: check if current variant is met more times than the one found so far,
            # if so, reassign max_occurences and index variables
            # 3.1 if max_gram != 0: if at least 1 context+variant is met in the corpus, then choose the one variants[index]
            # (with the biggest types_of_grams and occurences)
            # 3.2 otherwise, context+variable doesn't exist in the corpus and correct it with usual Norwig's correction()

            max_gram = 0
            max_occurences = 0
            index = 0

            # iterate over all variants
            for j, variant in enumerate(variants):
                # concatenate the context (previous words) and the current variant
                variant_sentence = context+[variant]

                # see the self.find_grams_freq method description bellow ->
                types_of_grams, occurences = self.find_grams_freq(variant_sentence)
                

                if types_of_grams == 0:
                    # this context+variant isn't met in the corpus
                    continue
                
                if types_of_grams > max_gram:
                    max_gram = types_of_grams
                    max_occurences = occurences
                    index = j
                if types_of_grams == max_gram:
                    if max_occurences < occurences:
                        max_occurences = occurences
                        index = j

            if max_gram != 0:
                correct_word = variants[index]
            else:
                correct_word = correction(word)

            self.result.append(correct_word)
        self.result = ' '.join(self.result)
        return self.result


    def check_1st_word(self, word: str):
        """ Method for selection of the correct 1st word, if it is misspelled
        Args:
            word (str): the 1st word in the sentence
        Returns: None
            depending on the analysis, the method correct the 1st word
            and appends it to the result
        """
        # get available variants for this word and sort them by frequency
        variants = self.get_variants(word)
        variants = sorted(list(variants), key=P, reverse=True)
        # if there is no available variants return the same word
        if len(variants) == 0: 
            self.result.append(word)
            return

        # iteratively check each of them
        for variant in variants:
            # select the correct word based on expected tag
            # use the tagged current sentence from which the word is taken
            # and form a new "corrected" sentence with the current variant
            expected_tag = self.tagged[0].dep_
            variant_sentence = variant + ' '.join(self.words[1:])

            # if the current variant suits, based on expected POS tag, then ...
            ok = self.check_tag_accordance(variant_sentence, expected_tag, 0)

            if ok:
                # ... select this one
                # this method allows to combine Norvig's correction
                # and filter unsuitable variants via POS tags
                self.result.append(variant)
                return
            
        # if we have some variants but none of them is assigned as an expected tag,
        # return variants[0] - the same as a basic Norvig's correction()
        self.result.append(variants[0])


    def check_tag_accordance(self, sentence: str, expected_tag, i: int) -> bool:
        """ Method for checking do an expected tag and variant's tag coincide
        Args:
            sentence (str): sentence with inserted variant word instead of misspelled one
            expected_tag (Token): tag of a misspelled, gotten from the initial sentence (with a mistake)
            i (int): index of a misspelled word
        Returns:
            bool: coincide or not
        """
        tokens = self.nlp(sentence)
        return tokens[i].dep_ == expected_tag


    def get_variants(self, word: str) -> list[str]:
        """ Method for generating available candidates as a correct word instead of a misspelled word
        Args:
            word (str): misspelled word
        Returns:
            list[str]: list of candidates
        """
        return known(edits1(word)) or known(edits2(word))
    

    def find_grams_freq(self, sequence: list[str], check: bool=False) -> tuple[int, int]:
        """ Method for finding existing n_grams in the corpus that correspond to the current variant.
        Args:
            sequence (list[str]): list of words: context + possibly correct word
        Returns:
            : tuple[int, int] where tuple[0] is n in n_gram and 
                tuple[1] is amount of this n_gram is met in the corpus
        Example: 
            find_grams_freq(['a', 'babe', 'in', 'the',	'woods']) => 
            frequencies = [(2, 8805), (3, 630), (4, 16), (5, 16)] that means: 
                the bigram ('the', 'woods') is met in the vocab 8805 times, the trigram ('in', 'the', 'woods')
                is met in the vocab 630 times, etc.
            summation = 9467 = 8805+630+16+16
            types_of_grams = len(frequencies) that means
                in how many types of n_grams (bigrams/trigrams/fourgrams/fivegrams) within the current context
                the current possibly correct word is met in the corpus.
        """
        frequences = []
        summation = 0

        # cut the sequence to 5 words because we have maximum fivegrams
        sequence = sequence[-5:]
        for i in range(2,len(sequence)+1):
            current_context = tuple(sequence[-i:])
            if current_context in bigrams.keys():
                frequences.append((i, bigrams[current_context]))
            elif current_context in threegrams.keys():
                frequences.append((i, threegrams[current_context]))
            elif current_context in fourgrams.keys():
                frequences.append((i, fourgrams[current_context]))
            elif current_context in fivegrams.keys():
                frequences.append((i, fivegrams[current_context]))

        summation = sum([x[1] for x in frequences])
        if check:
            return frequences, summation
        
        types_of_grams = len(frequences)
        return types_of_grams, summation

        

In [4]:
corrector = Corrector()
corrector.find_grams_freq(['a', 'babe', 'in', 'the',	'woods'], check=True)

([(2, 8805), (3, 630), (4, 16), (5, 16)], 9467)

In [5]:
corrector.correct_sentence("I can't belive how bueatiful the sunset was yesterday.")

'i can t believe how beautiful the sunset was yesterday'

In [6]:
corrector.correct_text("I red the book yesturday and it was really interessing. I wantf \
                       to telk about it to all my frends.", return_list=False)

'i red the book yesterday and it was really interesting. i want to tell about it to all my friends'

---

# Justification

Firstly, I red the paper with the Norvig's solution and highlighted several points: it's based on the frequency in the corpus and considers only grammar typos. His solution doesn't take into account keybord layout and error that might appear because of the incorrect finger position.

Secondly, I watched through the provided datasets. There are bigrams.txt, fivegrams.txt, and cocal_all_links.txt with bigrams and corresponding POS tags.

I decided to improve the Norvig's solution using these n_grams. While I have fivegrams, I can create threegrams and fourgrams extracting the required amount of words from fivegrams. Doing this, I can achieve better performance, I can get more information about dependencies in the text. However, there is always a problem with lack of data. Although, I have 1m of fivegrams, the language is too flexible to be enclosed in a million of rows. Also, we don't know the position of those fivegrams in sentences. It would give much more information about how texts start and end. I mentioned this, because there is a problem with fixing some typos in the 1st words in sentences. There is no previous context for the 1st words and we rely only on Norvig's correction without n_grams.

>I tried to deal with this problem by looking for required POS tags among variants. 

However, I saw that the 1st words are mostly 'There', 'We', 'I', 'The', 'Me', '“Jane', 'Be', 'It', - short and sometimes bring no meaning (determiners). It is easy to assume that if a short word is misspelled there are too much variants per correction. Thus, this problem might be solved with sentence analysis and more advanced application of POS tags while Norvig's and my approaches give only ~ 45% of accuracy and some similar results:
```
(Case 3): correct word / corrected word
i is
they then
the the
i o
the the
the the
so to
each each
``` 

>I improved text annalysis using splitting fivegrams to 3-4-5grams.

I assumed that words appearing in a bigger number of various contexts are supposed to be used more frequently. After generating possible corrections for a misspelled word, I consider these options in the context provided. A context+variant pair that appears in the largest number of n_grams and most frequently is chosen. Thus, unlike Norvig’s algorithm, this approach takes into account not only the frequency of occurrence of a word in the corpus as a whole, but also the frequency of its occurrence in the current context.


**Testing**

For the test part I downloaded the "Jane Eyre" book in the same format as "Sherlock Holmes" book in the 'big.txt' file. I preprocessed file a little bit and left only the chapters' text: removed the head with titles, authors, edition, preface, etc. Then I processed the file into the string variable and removed some unnecessary titles and punctuation (chapters allignment and several empty lines). As a result, I got a list of clear sentences. Then I synthetically generated mistakes in words using edit1 and edit2 types of typos (with 50% probability per each).

I tested algorithms via:
1) Spoiling the last words in sentences and fixing them only.
2) Spoiling 50% of words and fixing the whole sentence.

In the 1st approach I got:
```
My solution: 0.716
Norvig's solution 0.664
```
To sum up, this approach demonstrates 5.2% of increased accuracy.

In the 2nd approach my solution's accuracy is bigger than Norvig's by 1.5%. 

---

## Test My and Norwigs' solution

### Test set generation

In [7]:
test = ''
with open('useful data/pg1260.txt', 'r') as f:
    for line in f:
        test += line

test = re.sub(r'\n\n\n\n\n\n', ' ', test)
test = re.sub(r'\n\n\n\n\n', ' ', test)
test = re.sub(r'\n\n\n\n', ' ', test)
test = re.sub(r'\n\n\n', ' ', test)
test = re.sub(r'\n\n', ' ', test)
test = re.sub(r'\n', ' ', test)
test = re.sub(r'\ufeff', '', test)
test = re.sub(r'CHAPTER \w ', '', test)
test = re.sub(r'CHAPTER \w\w ', '', test)
test = re.sub(r'CHAPTER \w\w\w ', '', test)
test = re.sub(r'CHAPTER \w\w\w\w ', '', test)
test = re.sub(r'CHAPTER \w\w\w\w\w ', '', test)
test = re.sub(r'CHAPTER \w\w\w\w\w\w ', '', test)
test = test.split('. ')
test[:5]

['There was no possibility of taking a walk that day',
 'We had been wandering, indeed, in the leafless shrubbery an hour in the morning; but since dinner the cold winter wind had brought with it clouds so sombre, and a rain so penetrating, that further outdoor exercise was now out of the question',
 'I was glad of it: I never liked long walks, especially on chilly afternoons: dreadful to me was the coming home in the raw twilight, with nipped fingers and toes, and a heart saddened by the chidings of Bessie, the nurse, and humbled by the consciousness of my physical inferiority to Eliza, John, and Georgiana Reed',
 'The said Eliza, John, and Georgiana were now clustered round their mama in the drawing-room: she lay reclined on a sofa by the fireside, and with her darlings about her (for the time neither quarrelling nor crying) looked perfectly happy',
 'Me, she had dispensed from joining the group; saying, “She regretted to be under the necessity of keeping me at a distance; but that u

In [15]:
test = test[:500]
test_copy = test.copy()[:200]

In [9]:
for i in range(20):
    print(test[i].split()[0], end=', ')

There, We, I, The, Me,, “Jane,, Be, It, I, Folds, At, Afar,, I, They, The, I, The, The, So, Each, 

### Case 1: errors in the end

In [28]:
# =================================================================
# Case 1:
# the only mistake is in the last word
# =================================================================

import random
random.seed(1)

def make_mistake(words: list[str], return_list: bool=True, position: int=-1):
    last_word = words[position]
    # choose the mistakes type
    choice = random.randint(0, 1)
    if choice == 0:
        mistake_words = edits1(last_word)
    else:
        mistake_words = edits2(last_word)
    # randomly select one incorrect word and replace it
    incorrect_word = random.choice(list(mistake_words))
    words[position] = incorrect_word

    # return the required type
    if return_list: return words
    return " ".join(words)

In [29]:
# =================================================================
# CHECK MY SOLUTION
# =================================================================

count = 0
total = 0
corrector = Corrector()
for sentence in test:
    # iteratively go through the test set
    if sentence == '':
        continue
    total += 1
    # get separate words from a sentence and extract the last one
    words = get_words(sentence)
    correct_word = words[-1]
    # add mistakes and return the string
    incorrect_sentence = make_mistake(words, return_list=False)
    # fix mistakes
    corrected_word = corrector.correct_sentence(incorrect_sentence).split()[-1]
    # compare
    if corrected_word == correct_word:
        count += 1
my_result = count / (total)


# =================================================================
# CHECK NORVIG'S SOLUTION
# =================================================================

count = 0
total = 0
for sentence in test:
    if sentence == '':
        continue
    total += 1
    # get separate words and the last word
    words = get_words(sentence)
    correct_word = words[-1]
    # add a mistake to the last word
    incorrect_sentence = make_mistake(words)
    # correct mistakes in the last word using Norvig's solution
    corrected_word = correction(incorrect_sentence[-1])
    if corrected_word == correct_word:
        count += 1
norvig_result = count / (total)


print('My solution:', my_result)
print("Norvig's solution", norvig_result)

My solution: 0.716
Norvig's solution 0.664


### Case 2: 50% words have errors

In [33]:
# =================================================================
# Case 2:
# The same data from Jane Eyre book
# mistakes are among the sentences, about 50% of words are incorrect
# =================================================================

def make_several_mistakes(words: list[str], return_list: bool=True):
    incorrect = words.copy()
    # misspell 50% of words
    for i,word in enumerate(incorrect):
        if random.randint(0, 1) == 0:
            # then misspell
            # type of mistake:
            choice = random.randint(0, 1)
            if choice == 0:
                mistake_words = edits1(word)
            else:
                mistake_words = edits2(word)
            incorrect_word = random.choice(list(mistake_words))
            incorrect[i] = incorrect_word

    if return_list: return incorrect
    return ' '.join(incorrect)


def apply_norvig_correction(words: list[str]):
    # method for Norvig's correction of the whole sentence
    corrected = []
    for word in words:
        if word in WORDS:
            corrected.append(word)
            continue
        corrected.append(correction(word))
    return corrected


In [34]:
# =================================================================
# CHECK MY SOLUTION
# =================================================================
random.seed(10)

count = 0
total = 0
corrector = Corrector()
for sentence in test_copy:
    if sentence == '':
        continue
    words = get_words(sentence)
    incorrect_sentence = make_several_mistakes(words, return_list=False)
    corrected_sentence = corrector.correct_sentence(incorrect_sentence).split()

    for correct, corrected in zip(words, corrected_sentence):
        total += 1
        if correct == corrected:
            count += 1
my_result = count / total


# =================================================================
# CHECK NORVIG'S SOLUTION
# =================================================================
count = 0
total = 0
for sentence in test_copy:
    if sentence == '':
        continue
    words = get_words(sentence)
    correct_sentence = ' '.join(words)
    incorrect_sentence = make_several_mistakes(words)
    corrected_sentence = apply_norvig_correction(incorrect_sentence)

    for correct, corrected in zip(words, corrected_sentence):
        total += 1
        if correct == corrected:
            count += 1
norvig_result = count/total


print('My solution:', my_result)
print("Norvig's solution:", norvig_result)


My solution: 0.8231830156516349
Norvig's solution: 0.8135750813575081


### Case 3: errors in the 1st words

In [30]:
# =================================================================
# Case 3: 1st words have error
# the only mistake is in the first word
# =================================================================

random.seed(10)

# =================================================================
# CHECK MY SOLUTION
# =================================================================
count = 0
total = 0
corrector = Corrector()
for sentence in test:
    # iteratively go through the test set
    if sentence == '':
        continue
    total += 1
    # get separate words from a sentence and extract the last one
    words = get_words(sentence)
    correct_word = words[0]
    # add mistakes and return the string
    incorrect_sentence = make_mistake(words, return_list=False, position=0)
    # fix mistakes
    corrected_word = corrector.correct_sentence(incorrect_sentence).split()[0]
    # compare
    if corrected_word == correct_word:
        count += 1
my_result = count / total


# =================================================================
# CHECK NORVIG'S SOLUTION
# =================================================================

count = 0
total = 0
for sentence in test:
    if sentence == '':
        continue
    total += 1
    # get separate words and the last word
    words = get_words(sentence)
    correct_word = words[0]
    # add a mistake to the last word
    incorrect_sentence = make_mistake(words, position=0)
    # correct mistakes in the last word using Norvig's solution
    corrected_word = correction(incorrect_sentence[0])
    if corrected_word == correct_word:
        count += 1
norvig_result = count / total


print('My solution:', my_result)
print("Norvig's solution", norvig_result)

My solution: 0.438
Norvig's solution 0.484


#### Useful resources (also included in the archive in moodle):

1. [Possible dataset with N-grams](https://www.ngrams.info/download_coca.asp)
2. [Damerau–Levenshtein distance](https://en.wikipedia.org/wiki/Damerau–Levenshtein_distance#:~:text=Informally%2C%20the%20Damerau–Levenshtein%20distance,one%20word%20into%20the%20other.)